#Code implementation of the paper "To Act or React?"
https://arxiv.org/pdf/1906.11932v1.pdf

Connect: ahmedrashidqazi@gmail.com

###Libraries

In [78]:
import os
import sys
import pandas as pd
import numpy as np
import sklearn
from google.cloud import bigquery
import re
import random

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
client = bigquery.Client('ffh-bigquery')

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Data
##Time Frame - 01/2015 - 10/2018

###DA = Most Active 3 thousand Subreddits based on the average number of posts per month

In [5]:
DA = []
DA = pd.read_csv("/content/gdrive/MyDrive/Most Active 3000 Subreddit Names.csv")
DA

,subreddit,a
0,AskReddit,210116.4
1,AutoNewspaper,183784.6
2,RocketLeagueExchange,146558.1
3,CircleofTrust,142087.7
4,StocksInFocus,128409.5
...,...,...
2995,Rainmeter,674.0
2996,funwithwords,673.4
2997,Ooer,673.1
2998,u_b206050de3,673.0


###DB = 38 Quarantined/Banned Subreddits

In [6]:
DB = ['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal',
 'altright',
 'Incels'] 

####The code below was used to extract the above 36 subreddits

In [7]:
# DB = []

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.2015_12  WHERE quarantine = true """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   x = x['subreddit']
#   print(x.unique())
#   for k in x.unique():
#     if k not in DB:
#       DB.append(k)

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# years = ['2018_08', '2019_01']
# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT subreddit FROM fh-bigquery.reddit_posts.""" + year + """ WHERE quarantine = true """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       x = x['subreddit']
#       print(x.unique())
#       for k in x.unique():
#         if k not in DB:
#           DB.append(k)
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break


# DB.append("WatchPeopleDie")
# DB.append("Physical_Removal")

In [8]:
print("Total number of banned/quarantined subreddits", len(DB))
DB

Total number of banned/quarantined subreddits 38


['Braincels',
 'TheRedPill',
 'Ice_Poseidon',
 'CringeAnarchy',
 'TimeToGo',
 'Ice_Poseidon2',
 'DebateAltRight',
 'OffensiveMemes',
 'thinspocommunity',
 'ZOG',
 'EDFood',
 'spacedicks',
 'whitebeauty',
 'Clown__World',
 'Clownworldwar',
 'FULLCOMMUNISM',
 'GentilesUnited',
 'NorthwestFront',
 'okbuddyanarchy',
 'fragilejewishredditor',
 'Identitarians',
 'niggas',
 'holocaust',
 'tha_pit_pit',
 'Mayo_town',
 'CringeChaos',
 'kangznsheeit',
 'cursedx100images',
 'SubforWhitePeopleOnly',
 'cursedx3images',
 'The_Donald',
 'ChapoTrapHouse',
 'AmericanJewishPower',
 'honkler',
 'WatchPeopleDie',
 'Physical_Removal',
 'altright',
 'Incels']

###DH = 117 Hateful Subreddits

In [9]:
DH = ['news',
 'conspiracy',
 'kotakuinaction',
 'pics',
 'legaladvice',
 'drama',
 'politics',
 'subredditdrama',
 'askreddit',
 'videos',
 'movies',
 'announcements',
 'fatpeoplehate',
 'gaming',
 'games',
 'christianity',
 'worldnews',
 'pcgaming',
 'livestreamfail',
 'conservative',
 'bitcoin',
 'television',
 'libertarian',
 'europe',
 'topmindsofreddit',
 'subredditoftheday',
 'socialism',
 'pussypassdenied',
 'morbidquestions',
 'jordanpeterson',
 'greatawakening',
 'blackpeopletwitter',
 'aww',
 'wallstreetbets',
 'unpopularopinion',
 'uncensorednews',
 'outoftheloop',
 'islam',
 'askhistorians',
 'publicfreakout',
 'european',
 'subredditcancer',
 'mensrights',
 'h3h3productions',
 'enoughtrumpspam',
 'all',
 'tumblrinaction',
 'pussypass',
 'metacanada',
 'latestagecapitalism',
 'imgoingtohellforthis',
 'canada',
 'asktrumpsupporters',
 'againsthatesubreddits',
 '4chan',
 'thenewright',
 'sjwhate',
 'milliondollarextreme',
 'gendercritical',
 'frenworld',
 'easternsunrising',
 'catholicism',
 'anarcho_capitalism',
 'iama',
 'relationships',
 'overwatch',
 'adviceanimals',
 'ps4',
 'leagueoflegends',
 'hiphopheads',
 'funny',
 'rickandmorty',
 'parenting',
 'nfl',
 'justneckbeardthings',
 'gamingcirclejerk',
 'todayilearned',
 'sandersforpresident',
 'nsfw_gif',
 'neoliberal',
 'mapporn',
 'foodporn',
 'food',
 'amitheasshole',
 'soccer',
 'sex',
 'punchablefaces',
 'makeupaddiction',
 'magictcg',
 'houston',
 'hearthstone',
 'dnd',
 'battlefield',
 'askmen',
 'whiterights',
 'publichealthwatch',
 'europeannationalism',
 'israel',
 'coontown',
 'chadright',
 'truecels',
 'mgtow',
 'weekendgunnit',
 'mr_trump',
 'the_donalds',
 'worldpolitics',
 'nationalsocialism',
 'aznidentity',
 'mongoloidsupremacy',
 'the_adolf',
 't_d',
 'samandtolki',
 'shitpoliticssays',
 'rightwinglgbt',
 'incelabode',
 'europeans',
 'commonfilth']

####Following code was used to extract the above 117 hateful subreddits

#####Extracting the text, title, score and number of comments from the subreddit r/AgainstHateSubreddits

In [10]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'AgainstHateSubreddits') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'AgainstHateSubreddits' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'AgainstHateSubreddits' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv", index = False)

#####Extracting the text, title, score and number of comments from the subreddit r/SubredditDrama

In [11]:
# text = pd.DataFrame([], columns = ['selftext', 'title'])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.full_corpus_201512  WHERE STRING(TIMESTAMP_SECONDS(created_utc)) LIKE '%2015%' AND (subreddit = 'SubredditDrama') """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.2015_12  WHERE subreddit = 'SubredditDrama' """ 
# x = client.query(query).to_dataframe()
# if len(x) > 0: 
#   text = pd.concat([text,x])

# years = ['2016_01', '2017_01', '2018_01', '2019_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT selftext, title, score, num_comments FROM fh-bigquery.reddit_posts.""" + year + """ WHERE subreddit = 'SubredditDrama' """ 
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2019_09" or year == "2018_13":
#       break

# text.to_csv("/content/gdrive/MyDrive/SubredditDramaText.csv", index = False)

#####Analysing the subreddit to extract names of hateful subreddits from AgainstHateSubreddits

In [12]:
AHS = pd.read_csv("/content/gdrive/MyDrive/AgainstHateSubredditsText.csv")
AHS

,selftext,title,score,num_comments
0,NaN,/r/WorldPolitics now linking ONLY to anti-immi...,74.0,30.0
1,NaN,A hate subreddit brigade.,45.0,16.0
2,NaN,Action Taken by mods - Claims that /r/Europe i...,30.0,7.0
3,NaN,"""We need to start a trend on twatter #RemoveKe...",10.0,3.0
4,NaN,Test post please ignore,1.0,1.0
...,...,...,...,...
22973,[removed],What the actual f*ck is this sub,1.0,0.0
22974,[removed],Some thoughts I've got from being on this sub,1.0,0.0
22975,[removed],META: Should direct reddit links be banned on ...,18.0,8.0
22976,[removed],Pro-China Protestors Nazi salute in Vancouver ...,1.0,0.0


In [13]:
#Posts below a score of 100 will be dropped
AHS = AHS.sort_values(by = "score", ascending = False)
AHSNew = AHS[(AHS['score'] > 5) & (AHS['num_comments'] > 5)]
AHSNew['selftext'] = AHSNew['selftext'].fillna(" ")
AHSNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
7380,,The ACTUAL FUCKING NAZIS AND WHITE NATIONALIST...,43178.0,2141.0
14743,[deleted],"The_Donald is a violent, extremist group being...",38130.0,2071.0
6231,,HanAssholeSolo wished for people to be doxxed ...,30570.0,1710.0
10093,"For those who may not be aware, /r/The_donald,...",Reddit admins have confirmed they are comforta...,30338.0,2017.0
5426,,/r/The_Donald Has Built A Document With The Ad...,28866.0,2375.0
...,...,...,...,...
564,,"""Tactics used by cancerous elements to stop fr...",6.0,9.0
16658,,"Moderators at r/azula refuse to remove porn ""a...",6.0,11.0
523,,On African Americans threatening to move to Ge...,6.0,6.0
1167,,"""Time to remove [Muslims] by any means people....",6.0,13.0


#####Analysing the subreddit to extract names of hateful subreddits from SubredditDrama

In [14]:
SD = pd.read_csv("/content/gdrive/MyDrive/SubredditDramaText.csv")
SD

,selftext,title,score,num_comments
0,NaN,Should someone kill themselves if they have a ...,37.0,97.0
1,NaN,2XC debates with OP as to whether a women only...,158.0,447.0
2,NaN,/r/occult is arguing about something. I have n...,1274.0,473.0
3,NaN,Refugee Drama in r/Islam,10.0,56.0
4,NaN,/u/bestofoutrageculture stickies a post to the...,248.0,209.0
...,...,...,...,...
70584,[deleted],r/AskTeenGirls mod having a fit over guys answ...,3.0,8.0
70585,[removed],r/Dota2 Mods turn off comments after Chinese c...,1.0,4.0
70586,[removed],FYI r/atheism is handing out lifetime bans for...,1.0,5.0
70587,[removed],R/History Perma Banned for asking about Moors,0.0,28.0


In [15]:
#Posts below a score of 100 will be dropped
SD = SD.sort_values(by = "score", ascending = False)
SDNew = SD[(SD['score'] > 400) & (SD['num_comments'] > 200)]
SDNew['selftext'] = SDNew['selftext'].fillna(" ")
SDNew

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,selftext,title,score,num_comments
68129,/r/The_Donald has been quarantined. Discuss th...,/r/The_Donald has been quarantined. Discuss th...,65481.0,24078.0
60526,,r/fuckthealtright mod made a detailed post of ...,51349.0,3311.0
53887,"It's literally the ENTIRE comment section, but...","r/The_Donald is imploding, following Trump's p...",41074.0,6525.0
44455,The_Donald has [gone private](/r/the_donald). ...,The_donald has gone private in protest of thei...,39328.0,10391.0
41072,/r/altright has been banned by the admins for ...,/r/altright has been banned. Discuss this here!,34168.0,8274.0
...,...,...,...,...
66666,,Is it alright for a female jew to write contov...,401.0,259.0
25407,,"Admins ask /r/guns to remove sidebar picture, ...",401.0,671.0
54811,https://www.reddit.com/r/worstof/comments/8209...,Users in /r/WorstOf try to figure out what dox...,401.0,389.0
41443,,Nicki Minaj's ex makes homeless man do pushups...,401.0,259.0


#####FIinding reddit name matches from the text

In [16]:
def clean_string(st):
  st = st.lower()
  st = st.replace(",","")
  st = st.replace(".","")
  st = st.replace(":","")
  st = st.replace("!","")
  st = st.replace("'","")
  st = st.replace(")","")
  st = st.replace("(","")
  st = st.replace("[","")
  st = st.replace("]","")
  st = st.replace("https","")
  return st


def find_matches(text):
  matches = {}
  for i in range(0, len(text)):
    m = []
    x = text.iloc[i]

    y = re.findall(r"/r/([^\s/]+)", x['title'])
    if len(y) > 0:
      for i in y:
        temp = clean_string(i)
        m.append(temp)

    z = re.findall(r"/r/([^\s/]+)", x['selftext'])
    if len(z) > 0:
      for i in z:
        temp = clean_string(i)
        m.append(temp)

    for i in m:
      if i not in matches.keys():
        matches[i] = 1
      else:
        matches[i] = matches[i] + 1
      break
  
  print("Matches found:", len(matches))
  print(matches)
  print("\n\n")

  temp_db = []
  for i in DB:
    temp_db.append(i.lower())

  matcheswithdb = []
  for i in matches.keys():
    if i in temp_db:
      matcheswithdb.append(i)

  print("Matches also found in quarantined/banned subreddits:", len(matcheswithdb))
  print(len(matcheswithdb), matcheswithdb)
  print("\n\n")

  print("Removing matches found in DB")
  for i in matcheswithdb:
    matches.pop(i)

  print("Final matches:", len(matches))
  print(matches)
  return matches

In [17]:
AHS_Final = find_matches(AHSNew)

Matches found: 537
{'the_donald': 388, 'physical_removal': 39, 'altright': 80, 'incels': 59, 'imgoingtohellforthis': 16, 'uncensorednews': 99, 'onguardforthee': 3, 'whiterights': 37, 'waterniggas': 1, 'blog': 1, 'greatawakening': 10, 'milliondollarextreme': 55, 'all': 9, 'conservative': 30, 'subredditdrama': 1, 'honkler': 4, 'race_realism': 1, 'obese_niggers': 1, 'topmindsofreddit': 2, 'shitpoliticssays': 6, 'metacanada': 72, 'conspiracy': 126, 'kiachatroom': 1, 'landofbuddies': 1, 'offensivespeech': 4, 'chadfish': 2, 'politics': 8, 'cringanarchy': 1, 'games': 1, 'canada': 22, 'stonetoss': 2, 'majorityrights': 1, 'drama': 13, 'antifa': 4, 'sjwhate': 10, 'clownworldwar2': 1, 'askthedonald': 1, 'racialrealism': 1, 'hillaryforprison': 4, 'catholicism': 11, 'israel': 18, 'rightwinglgbt': 6, 'clownworldwar': 8, 'blackcrime': 2, 'cringeanarchy': 73, 'modnews': 1, 'justiceserved': 2, 'transgenderkids': 1, 'the_donaldunleashed': 3, 'debatefascism': 3, 'againsthatesubreddits': 18, 'incest': 1, 

In [18]:
SD_Final = find_matches(SDNew)

Matches found: 714
{'the_donald': 45, 'altright': 3, 'trumpgret': 2, 'videos': 22, 'theredpill': 5, 'watchpeopledie': 2, 'incels': 2, 'physical_removal': 2, 'nfl': 8, 'teenagers': 3, 'gaming': 16, 'smashbros': 4, 'chapotraphouse': 7, 'conspiracy': 48, 'frenworld': 1, 'worldnews': 14, 'asktrumpsupporters': 2, 'politics**': 1, 'uncensorednews': 5, 'leagueoflegends': 9, 'battlefield': 6, 'wikileaks': 2, 'unexplainedphotos': 1, 'makenewfriendshere': 1, 'games': 15, 'amitheasshole': 7, 'kotakuinaction': 41, 'cringeanarchy': 9, 'rickandmorty': 8, 'iamatotalpieceofshit': 4, 'joerogan': 2, 'pizzagate': 1, 'shoplifting': 2, 'television': 8, 'pics': 37, 'pcgaming': 14, 'me_irl': 3, 'unpopularopinion': 5, 'deepfakes': 1, 'greatawakening': 6, 'hearthstone': 6, 'rainbow6': 3, 'fitgirls': 2, 'freefolk': 3, 'livestreamfail': 11, 'marchagainsttrump': 1, 'socialism': 6, 'memes': 1, 'goldtesting': 1, 'morbidquestions': 6, 'iama': 29, 'stopgaming': 1, 'coachella': 1, 'ice_poseidon2': 2, 'oculus': 3, 'sec

In [19]:
#sorting based on score
ahs = (sorted(AHS_Final.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))
sd = (sorted(SD_Final.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))

#####Extracting top hateful subreddits

In [20]:
final = []

#finding common ground
common_names = []
for i in sd:
  for j in ahs:
    if i[0]  == j[0]:
      x = (i[0], i[1]+j[1])
      common_names.append(i[0])
      final.append(x)

#adding the rest
for i in sd:
  if i[0] not in common_names:
    final.append(i)

for i in ahs:
  if i[0] not in common_names:
    final.append(i)

print("Length", len(final))
print(final)

Length 1096
[('news', 71), ('conspiracy', 174), ('kotakuinaction', 63), ('pics', 39), ('legaladvice', 38), ('drama', 46), ('politics', 39), ('subredditdrama', 29), ('askreddit', 26), ('videos', 23), ('movies', 20), ('announcements', 24), ('fatpeoplehate', 19), ('gaming', 17), ('games', 16), ('christianity', 17), ('worldnews', 30), ('pcgaming', 15), ('livestreamfail', 14), ('conservative', 41), ('bitcoin', 12), ('television', 9), ('libertarian', 11), ('europe', 16), ('topmindsofreddit', 8), ('subredditoftheday', 8), ('socialism', 9), ('pussypassdenied', 8), ('morbidquestions', 7), ('jordanpeterson', 12), ('greatawakening', 16), ('blackpeopletwitter', 9), ('aww', 8), ('wallstreetbets', 6), ('unpopularopinion', 17), ('uncensorednews', 104), ('outoftheloop', 6), ('islam', 22), ('askhistorians', 6), ('wtf', 5), ('publicfreakout', 10), ('iamatotalpieceofshit', 5), ('european', 185), ('blog', 5), ('upliftingnews', 4), ('subredditcancer', 9), ('shitredditsays', 4), ('roastme', 4), ('protectand

In [21]:
final2 = []

#picking the reddits with more than 5 mentions
for i in final:
  if i[1] > 5:
    final2.append(i)

print(len(final2))
print(final2)

117
[('news', 71), ('conspiracy', 174), ('kotakuinaction', 63), ('pics', 39), ('legaladvice', 38), ('drama', 46), ('politics', 39), ('subredditdrama', 29), ('askreddit', 26), ('videos', 23), ('movies', 20), ('announcements', 24), ('fatpeoplehate', 19), ('gaming', 17), ('games', 16), ('christianity', 17), ('worldnews', 30), ('pcgaming', 15), ('livestreamfail', 14), ('conservative', 41), ('bitcoin', 12), ('television', 9), ('libertarian', 11), ('europe', 16), ('topmindsofreddit', 8), ('subredditoftheday', 8), ('socialism', 9), ('pussypassdenied', 8), ('morbidquestions', 7), ('jordanpeterson', 12), ('greatawakening', 16), ('blackpeopletwitter', 9), ('aww', 8), ('wallstreetbets', 6), ('unpopularopinion', 17), ('uncensorednews', 104), ('outoftheloop', 6), ('islam', 22), ('askhistorians', 6), ('publicfreakout', 10), ('european', 185), ('subredditcancer', 9), ('mensrights', 9), ('h3h3productions', 7), ('enoughtrumpspam', 8), ('all', 12), ('tumblrinaction', 6), ('pussypass', 26), ('metacanada'

In [22]:
DH = []
for i in final2:
  DH.append(i[0])
DH

['news',
 'conspiracy',
 'kotakuinaction',
 'pics',
 'legaladvice',
 'drama',
 'politics',
 'subredditdrama',
 'askreddit',
 'videos',
 'movies',
 'announcements',
 'fatpeoplehate',
 'gaming',
 'games',
 'christianity',
 'worldnews',
 'pcgaming',
 'livestreamfail',
 'conservative',
 'bitcoin',
 'television',
 'libertarian',
 'europe',
 'topmindsofreddit',
 'subredditoftheday',
 'socialism',
 'pussypassdenied',
 'morbidquestions',
 'jordanpeterson',
 'greatawakening',
 'blackpeopletwitter',
 'aww',
 'wallstreetbets',
 'unpopularopinion',
 'uncensorednews',
 'outoftheloop',
 'islam',
 'askhistorians',
 'publicfreakout',
 'european',
 'subredditcancer',
 'mensrights',
 'h3h3productions',
 'enoughtrumpspam',
 'all',
 'tumblrinaction',
 'pussypass',
 'metacanada',
 'latestagecapitalism',
 'imgoingtohellforthis',
 'canada',
 'asktrumpsupporters',
 'againsthatesubreddits',
 '4chan',
 'thenewright',
 'sjwhate',
 'milliondollarextreme',
 'gendercritical',
 'frenworld',
 'easternsunrising',
 'ca

#Hypotheses 3

H3. (a) User participation in hateful or dangerous subreddits negatively changes the nature of their participation within the broader community and (b) this can be corrected by community bans
and quarantines. These hypotheses, if valid, will show that
community bans and quarantines as pre-emptive moderator
actions can reduce offensiveness within the broader community. If H3(a) is invalid, we will have shown that user
behavior in outside communities is not influenced by their
participation in hateful or dangerous communities. If H3(b)
is invalid, we will have shown that there is a need for more
nuanced pre-emptive interventions to mitigate the impact
of hateful and dangerous communities.

###Offensive Speech Classifier

https://github.com/t-davidson/hate-speech-and-offensive-language

In [23]:
!pip2 install vaderSentiment
!pip2 install textstat
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
#!python2 /content/gdrive/MyDrive/Classifier/classifier/classifier.py

     |████████████████████████████████| 133kB 4.1MB/s 
     |████████████████████████████████| 1.9MB 5.8MB/s 
  Created wheel for textstat: filename=textstat-0.5.7-cp27-none-any.whl size=17052 sha256=80240b423b683e7bf01a12eb74fb18de4677fce09dba23951068451226829756
  Stored in directory: /root/.cache/pip/wheels/05/74/e9/d30ccd4d768d28521a933ad3c6039da9ed435f8c60ce7701ed
  Created wheel for pyphen: filename=Pyphen-0.10.0-cp27-none-any.whl size=1883496 sha256=702c7d8aa04ea0fe8df4f741417e5ff9e890f00abd9a73de4c818f0783c5c6ae
  Stored in directory: /root/.cache/pip/wheels/78/e3/76/42853bf2f7573a72fb5fe377a8406c979a5479ad6930506cde
  Created wheel for repoze.lru: filename=repoze.lru-0.7-cp27-none-any.whl size=11131 sha256=570e5faba6a113d9bc1f7efd2cc8fc53044e9032d49b4b8a769e7a7382bad12b
  Stored in directory: /root/.cache/pip/wheels/80/c7/7e/74a37083827b36f0e87d60824124409b8cc88eea0940cada72
Successfully built textstat pyphen repoze.lru
[nltk_data] Downloading package stopwords to /root/nltk_d

True

###Making the Comments Data

#####Only comments with a score of 1000 or above were selected
Following code was used to extract the comments data

In [24]:
# text = pd.DataFrame([], columns = ['body', 'author', 'subreddit', 'timestamp'])

# years = ['2015_01','2016_01', '2017_01', '2018_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT body, author, subreddit, TIMESTAMP_SECONDS(created_utc) FROM fh-bigquery.reddit_comments.""" + year + """ WHERE score >= 1000 """
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2018_11":
#       break


# text = text.drop(['timestamp'], axis = 1)
# text = text.rename(columns={'f0_': 'timestamp'})
# text.to_csv("/content/gdrive/MyDrive/Comments.csv", index = False)

#####Loading comments data

In [25]:
comments = pd.read_csv("/content/gdrive/MyDrive/Comments.csv")
comments

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,body,author,subreddit,timestamp
0,Alternate ending to ratatouille :/,Eyams,WTF,2015-01-09 16:58:03+00:00
1,This is proven fake. Was a failed PR stunt for...,Jeffuary,WTF,2015-01-29 02:53:07+00:00
2,It is the thing from The Faculty. It can be k...,shazneg,WTF,2015-01-15 11:03:15+00:00
3,In a tissue you fucking animal.,mankind_is_beautiful,WTF,2015-01-14 11:11:51+00:00
4,You say that but this could easily be Wisconsin.,OldStyle76,WTF,2015-01-16 21:38:52+00:00
...,...,...,...,...
2295331,Libertarians who bristle at this need to stop ...,mattjh,WhitePeopleTwitter,2018-10-29 12:27:03+00:00
2295332,Wow that's depressing. Why not put the leftove...,Ziggythesquid,WhitePeopleTwitter,2018-10-03 13:52:17+00:00
2295333,"*arms crossed, eye roll, nose exhale*... Mmmhm...",Sthurlangue,WhitePeopleTwitter,2018-10-22 16:33:46+00:00
2295334,We're wondering the same thing Courteney...,Futharon,WhitePeopleTwitter,2018-10-17 21:50:55+00:00


####Running classifier on comments

In [26]:
comments.head()

,body,author,subreddit,timestamp
0,Alternate ending to ratatouille :/,Eyams,WTF,2015-01-09 16:58:03+00:00
1,This is proven fake. Was a failed PR stunt for...,Jeffuary,WTF,2015-01-29 02:53:07+00:00
2,It is the thing from The Faculty. It can be k...,shazneg,WTF,2015-01-15 11:03:15+00:00
3,In a tissue you fucking animal.,mankind_is_beautiful,WTF,2015-01-14 11:11:51+00:00
4,You say that but this could easily be Wisconsin.,OldStyle76,WTF,2015-01-16 21:38:52+00:00


In [27]:
# Classifier has been run and results have been saved
# !python2 /content/gdrive/MyDrive/Classifier/classifier/classifier.py

In [28]:
classified_comments = pd.read_csv("/content/gdrive/MyDrive/CommentsWClassifier.csv")[0:2295304]
classified_comments.drop(classified_comments.index[classified_comments['author'] == '[deleted]'], inplace = True)
classified_comments

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,body,author,subreddit,timestamp,predictions
0,Alternate ending to ratatouille :/,Eyams,WTF,2015-01-09 16:58:03+00:00,Neither
1,This is proven fake. Was a failed PR stunt for...,Jeffuary,WTF,2015-01-29 02:53:07+00:00,Neither
2,It is the thing from The Faculty. It can be k...,shazneg,WTF,2015-01-15 11:03:15+00:00,Neither
3,In a tissue you fucking animal.,mankind_is_beautiful,WTF,2015-01-14 11:11:51+00:00,Hate speech
4,You say that but this could easily be Wisconsin.,OldStyle76,WTF,2015-01-16 21:38:52+00:00,Neither
...,...,...,...,...,...
2295299,Equally frustrating is the focus on scoreboard...,smatchymo,WhitePeopleTwitter,2018-10-19 00:16:16+00:00,Neither
2295300,"The Babel-fish is real, all you need is your t...",Galaxy-Hitchhiker,WhitePeopleTwitter,2018-10-13 03:28:48+00:00,Neither
2295301,The last time I flew spirit I was watching the...,Jchriddy,WhitePeopleTwitter,2018-10-17 03:48:42+00:00,Neither
2295302,Kid Rock is maybe the most successful marketin...,Billy_T_Sherman,WhitePeopleTwitter,2018-10-16 04:49:04+00:00,Neither


####Recording user activity in DB and DH

In [29]:
usrDH = classified_comments.subreddit.isin(DH)
usrDH = classified_comments[usrDH]
usrDH

,body,author,subreddit,timestamp,predictions
10,Sigh this is the same logic that keeps me from...,MarlonBain,aww,2015-01-08 03:55:43+00:00,Neither
11,The referees handle the footballs during the g...,i_enjoy_lemonade,nfl,2015-01-21 04:00:41+00:00,Neither
22,If a wolf has not been seen in the area for 70...,OathOfFeanor,news,2015-01-03 17:24:46+00:00,Neither
23,Minnesota gets to front page for first time th...,JarkoStudios,news,2015-01-10 01:37:56+00:00,Neither
24,CNN.com reposting the satirical image this Fre...,Schizotron,news,2015-01-07 13:04:09+00:00,Neither
...,...,...,...,...,...
2292673,Now imagine how the Flash Wolves have felt yes...,Sesshoin,leagueoflegends,2018-10-16 16:12:23+00:00,Neither
2292674,C9 has soul.\n\nTL has drive.\n\n100T has Fort...,nolphins,leagueoflegends,2018-10-17 17:20:24+00:00,Neither
2292675,If Faker comes to the NA LCS the world will ex...,FinancialDestroyer,leagueoflegends,2018-10-21 22:29:03+00:00,Neither
2292676,Using My Hero Academia Soundtrack and showing ...,OneTwoTrickFour,leagueoflegends,2018-10-09 19:04:26+00:00,Neither


In [30]:
usrDB = classified_comments.subreddit.isin(DB)
usrDB = classified_comments[usrDB]
usrDB

,body,author,subreddit,timestamp,predictions
331875,"If trp does get banned, I want to thank every ...",Syriz_,TheRedPill,2015-06-10 19:23:37+00:00,Neither
419130,Id fuck the shit out of that slut pencil,360BRoScope420,CringeAnarchy,2015-10-27 14:44:49+00:00,Neither
425333,His parents made him wear that as a reminder t...,ElvisChrist6,CringeAnarchy,2015-10-31 16:22:02+00:00,Neither
438021,Anyone else hear a record scratch when they go...,Army0fMe,CringeAnarchy,2015-10-26 16:29:58+00:00,Neither
450511,Mom's a savage.,SaltySocks,CringeAnarchy,2015-11-22 18:09:46+00:00,Neither
...,...,...,...,...,...
2245122,Wtf? They seriously have the audacity to yell ...,Alchemist_XP,The_Donald,2018-10-16 21:40:52+00:00,Offensive language
2245163,Something both sides of politics should agree ...,Napoleon_icecream,The_Donald,2018-10-23 23:25:29+00:00,Neither
2245176,I have never seen the left respond so quickly ...,georock,The_Donald,2018-10-10 22:02:42+00:00,Neither
2245221,"""These poor refugees are just looking for a be...",dont_be_low_energy,The_Donald,2018-10-22 01:05:51+00:00,Neither


In [31]:
user_activity_DH = {}
user_activity_DB = {}

for index, row in usrDB.iterrows():
  sub = row['subreddit']
  usr = row['author']
  if sub in DB:
    if usr in user_activity_DB.keys():
      if sub not in user_activity_DB[usr].keys():
        user_activity_DB[usr][sub] = 1
      else:
        user_activity_DB[usr][sub] = user_activity_DB[usr][sub] + 1
    else:
      user_activity_DB[usr] = {sub: 1}

for index, row in usrDH.iterrows():
  sub = row['subreddit']
  usr = row['author']
  if sub in DH:
    if usr in user_activity_DH.keys():
      if sub not in user_activity_DH[usr].keys():
        user_activity_DH[usr][sub] = 1
      else:
        user_activity_DH[usr][sub] = user_activity_DH[usr][sub] + 1
    else:
      user_activity_DH[usr] = {sub: 1}

print(len(user_activity_DH))
print(len(user_activity_DB))

285580
6173


In [32]:
def sort_dict(d):
  dic = {}
  for i in d:
    freq = 0
    for j in d[i]:
      freq = freq + d[i][j]
    dic[i] = freq
  
  return dic

In [33]:
print(len(user_activity_DB))
freq_usrDB = sort_dict(user_activity_DB)
freq_usrDB = dict(sorted(freq_usrDB.items(), key=lambda x:x[1], reverse = True))
len(freq_usrDB)

6173


6173

In [34]:
print(len(user_activity_DH))
freq_usrDH = sort_dict(user_activity_DH)
freq_usrDH = dict(sorted(freq_usrDH.items(), key=lambda x:x[1], reverse = True))
len(freq_usrDH)

285580


285580

#Analysis


Attributes of the classified comments

In [35]:
total = len(classified_comments)
hate = len(classified_comments[classified_comments['predictions'] == "Hate speech"])
offend = len(classified_comments[classified_comments['predictions'] == "Offensive language"])
neither = len(classified_comments[classified_comments['predictions'] == "Neither"])

print("Total number of comments: ", total)
print("Number of comments classified as HATE SPEECH:", hate, "\t\t", round(hate/total*100, 2), " %") 
print("Number of comments classified as OFFENSIVE LANGUAGE:", offend, "\t", round(offend/total*100, 2), " %")
print("Number of comments classified as Neither:", neither, "\t\t", round(neither/total*100, 2), "%")

Total number of comments:  2243624
Number of comments classified as HATE SPEECH: 51228 		 2.28  %
Number of comments classified as OFFENSIVE LANGUAGE: 139808 	 6.23  %
Number of comments classified as Neither: 2052588 		 91.49 %


User activity in banned/quarantined subreddits

In [36]:
#Number of comments with individual banned/quarantined subreddit names by every user
print(list(user_activity_DB.items())[0:10])
print('\n')
#Number of comments by each user on banned/quarantinned subreddits
print(list(freq_usrDB.items())[0:10])

#All users with comments on banned/quarantinned subreddits
usrDB

[('Syriz_', {'TheRedPill': 1}), ('360BRoScope420', {'CringeAnarchy': 1}), ('ElvisChrist6', {'CringeAnarchy': 1}), ('Army0fMe', {'CringeAnarchy': 1}), ('SaltySocks', {'CringeAnarchy': 1}), ('ubermatze', {'CringeAnarchy': 1}), ('jakeyjake1990', {'CringeAnarchy': 1}), ('Just1morefix', {'CringeAnarchy': 3}), ('Dude4001', {'CringeAnarchy': 1}), ('Gondile', {'CringeAnarchy': 1})]


[('Mechanical-one', 24), ('Lithuim', 23), ('yiannopoulos_m', 22), ('Medically', 20), ('malikobama', 19), ('Tia00017', 18), ('NotNolan', 17), ('shadowman3001', 17), ('IAmTuckerCarlson', 16), ('Niekisch', 13)]


,body,author,subreddit,timestamp,predictions
331875,"If trp does get banned, I want to thank every ...",Syriz_,TheRedPill,2015-06-10 19:23:37+00:00,Neither
419130,Id fuck the shit out of that slut pencil,360BRoScope420,CringeAnarchy,2015-10-27 14:44:49+00:00,Neither
425333,His parents made him wear that as a reminder t...,ElvisChrist6,CringeAnarchy,2015-10-31 16:22:02+00:00,Neither
438021,Anyone else hear a record scratch when they go...,Army0fMe,CringeAnarchy,2015-10-26 16:29:58+00:00,Neither
450511,Mom's a savage.,SaltySocks,CringeAnarchy,2015-11-22 18:09:46+00:00,Neither
...,...,...,...,...,...
2245122,Wtf? They seriously have the audacity to yell ...,Alchemist_XP,The_Donald,2018-10-16 21:40:52+00:00,Offensive language
2245163,Something both sides of politics should agree ...,Napoleon_icecream,The_Donald,2018-10-23 23:25:29+00:00,Neither
2245176,I have never seen the left respond so quickly ...,georock,The_Donald,2018-10-10 22:02:42+00:00,Neither
2245221,"""These poor refugees are just looking for a be...",dont_be_low_energy,The_Donald,2018-10-22 01:05:51+00:00,Neither


User activity in hateful subreddits

In [37]:
#Number of comments with individual hateful subreddit names by every user
print(list(user_activity_DH.items())[0:10])
print('\n')
#Number of comments by each user on hateful subreddits
print(list(freq_usrDH.items())[0:10])

#All users with comments on hateful subreddits
usrDH

[('MarlonBain', {'aww': 1, 'movies': 1, 'politics': 2}), ('i_enjoy_lemonade', {'nfl': 18}), ('OathOfFeanor', {'news': 2, 'todayilearned': 2, 'funny': 1, 'movies': 1, 'aww': 3, 'television': 1, 'worldnews': 1, 'videos': 2, 'pics': 3}), ('JarkoStudios', {'news': 1}), ('Schizotron', {'news': 1}), ('Iyamnotdead', {'pics': 6, 'videos': 1, 'funny': 7}), ('LevioSaawwww', {'pics': 2}), ('Dr-Major', {'pics': 1}), ('Chippxero', {'pics': 1}), ('firetype', {'pics': 1})]


[('TooShiftyForYou', 2635), ('PainMatrix', 545), ('kayjay25', 475), ('dick-nipples', 466), ('Rooonaldooo99', 464), ('SaintVanilla', 414), ('BunyipPouch', 375), ('snotbag_pukebucket', 343), ('J4CKR4BB1TSL1MS', 299), ('viva_la_vinyl', 284)]


,body,author,subreddit,timestamp,predictions
10,Sigh this is the same logic that keeps me from...,MarlonBain,aww,2015-01-08 03:55:43+00:00,Neither
11,The referees handle the footballs during the g...,i_enjoy_lemonade,nfl,2015-01-21 04:00:41+00:00,Neither
22,If a wolf has not been seen in the area for 70...,OathOfFeanor,news,2015-01-03 17:24:46+00:00,Neither
23,Minnesota gets to front page for first time th...,JarkoStudios,news,2015-01-10 01:37:56+00:00,Neither
24,CNN.com reposting the satirical image this Fre...,Schizotron,news,2015-01-07 13:04:09+00:00,Neither
...,...,...,...,...,...
2292673,Now imagine how the Flash Wolves have felt yes...,Sesshoin,leagueoflegends,2018-10-16 16:12:23+00:00,Neither
2292674,C9 has soul.\n\nTL has drive.\n\n100T has Fort...,nolphins,leagueoflegends,2018-10-17 17:20:24+00:00,Neither
2292675,If Faker comes to the NA LCS the world will ex...,FinancialDestroyer,leagueoflegends,2018-10-21 22:29:03+00:00,Neither
2292676,Using My Hero Academia Soundtrack and showing ...,OneTwoTrickFour,leagueoflegends,2018-10-09 19:04:26+00:00,Neither


In [38]:
top_haters = []
for i in user_activity_DB.keys():
  x = usrDB[usrDB['author'] == i]
  x = x['predictions']
  y = x.value_counts(normalize=True) * 100
  if y.iloc[0] == 100:
    continue
  
  top_haters.append((len(x), i, y))

top_haters = sorted(top_haters)
top_hater_names = []

for i in top_haters:
  top_hater_names.append(i[1])

print(len(top_hater_names))
print(top_hater_names)

330
['0verloader', 'A_Rex', 'AdmiralAkbar1', 'AereBourne', 'AlexJonesHasAIDS', 'ArseKraken', 'Avery_Richman', 'BAGELmode', 'BalooRed', 'BeeVomitImHome', 'Best_Of_The_Midwest', 'Bren-Ten', 'CallMeTheEndOfDays', 'Campellarino', 'CantGrammarGood', 'Capitally', 'CaptainPepe420', 'Captian_Cocksmith', 'CharredHam', 'Cherubixcube', 'Corporal-Hicks', 'Darkmetroidz', 'DavidGolani11', 'DelusionalDeplorable', 'DennisOswine', 'DingoFixedMyWater', 'DisrespectfulNipples', 'Dragon_Maister', 'DudebroWhateverman', 'Easter_Bunny', 'FeralHushpuppy', 'FlareKR', 'FlightLevel390', 'Flywolfpack', 'Fna1', 'Germstore', 'Ghostofelviscostello', 'GiraffeFish', 'GodEmperorTrump1', 'Greatmambojambo', 'GtiKyle', 'GuyWithNoName67', 'HIGH_ENERGY_MEMES', 'HandInAssholesSulu', 'HmmWhatsThat', 'Horageous', 'IFIFIFOKIEDOKE', 'IPbans', 'I_TRY_TO_BE_POSITIVE', 'IpsoKinetikon', 'Jane_a_Navy_wife', 'JeffreyRamstein', 'Jorgwalther', 'Junosu', 'Justin_is_Fidels_Son', 'KBDs', 'KiA423469420', 'Kush-Cookies', 'LeFabulousFaggot', '

In [39]:
top_haters_2 = []
for i in top_hater_names:
  x = usrDH[usrDH['author'] == i]
  x = x['predictions']
  y = x.value_counts(normalize=True) * 100

  if x.empty == True or y.iloc[0] == 100:
    continue
  
  top_haters_2.append((len(x), i, y))

top_haters_2 = sorted(top_haters_2)
top_hater_names_2 = []

for i in top_haters_2:
  top_hater_names_2.append(i[1])

print(len(top_hater_names_2))
print(top_hater_names_2)

26
['FeralHushpuppy', 'Idontreadrepliesnoob', 'KBDs', 'wmegenney', 'RonDeGrasseDawtchins', 'Suzookus', 'bottomlines', 'theresheblowshard', 'wizardofthefuture', 'ArseKraken', 'Auphor_Phaksache', 'Mechanical-one', 'ghostmetalblack', 'PedroIsWatching', 'smackythefrog', 'CantGrammarGood', 'Greatmambojambo', 'RamblingMutt', 'G_Petronius', 'Captian_Cocksmith', 'HoundDogs', 'homefree122', 'AdmiralAkbar1', 'lets_get_hyyerr', 'erokk88', 'HighOnGoofballs']


In [40]:
top_haters_df = []

for i in range(0, len(top_hater_names_2)):
  x = classified_comments[classified_comments['author'] == top_hater_names_2[i]]
  y = x[x['predictions'] != 'Neither']
  if len(top_haters_df) == 0:
    top_haters_df = y
  else:
    top_haters_df = pd.concat([top_haters_df, y])

top_haters_final = top_haters_df['author'].unique()
print(top_haters_final)
top_haters_df

['FeralHushpuppy' 'Idontreadrepliesnoob' 'KBDs' 'wmegenney'
 'RonDeGrasseDawtchins' 'Suzookus' 'bottomlines' 'theresheblowshard'
 'wizardofthefuture' 'ArseKraken' 'Auphor_Phaksache' 'Mechanical-one'
 'ghostmetalblack' 'PedroIsWatching' 'smackythefrog' 'CantGrammarGood'
 'Greatmambojambo' 'RamblingMutt' 'G_Petronius' 'Captian_Cocksmith'
 'HoundDogs' 'homefree122' 'AdmiralAkbar1' 'lets_get_hyyerr' 'erokk88'
 'HighOnGoofballs']


,body,author,subreddit,timestamp,predictions
809674,First in line to vote. The poll workers litera...,FeralHushpuppy,The_Donald,2016-11-08 12:02:33+00:00,Offensive language
867634,Yeah. When I was a young adult (see: 2004 - 20...,FeralHushpuppy,news,2017-01-05 19:26:45+00:00,Hate speech
655330,Umm. . .so male?,Idontreadrepliesnoob,TumblrInAction,2016-07-06 03:46:32+00:00,Offensive language
656285,"I mean, I can *kinda* see it. . .",Idontreadrepliesnoob,justneckbeardthings,2016-07-15 05:08:05+00:00,Hate speech
715761,"This is how it starts, folks.",Idontreadrepliesnoob,CringeAnarchy,2016-09-19 14:50:48+00:00,Offensive language
...,...,...,...,...,...
1921846,"I wonder if it would work as a wake-up call, o...",HighOnGoofballs,niceguys,2018-06-25 00:21:11+00:00,Offensive language
2014334,I tried this and my dog ignored it and I ended...,HighOnGoofballs,aww,2018-07-22 02:07:16+00:00,Offensive language
2194839,It’s like ten thousand spoons when all you nee...,HighOnGoofballs,trashy,2018-09-22 22:36:34+00:00,Offensive language
2209126,"I see Judy Blume about once a month, she’s sup...",HighOnGoofballs,MovieDetails,2018-09-24 23:32:12+00:00,Offensive language


In [41]:
f1 = str(tuple(top_haters_final))[1:-1]
f2 = str(tuple(DH))[1:-1]
f3 = str(tuple(DB))[1:-1]
f4 = f2 + " , " + f3

In [42]:
# text = pd.DataFrame([], columns = ['body', 'author', 'subreddit', 'timestamp'])

# years = ['2015_01','2016_01', '2017_01', '2018_01']

# for i in years:
#   year = i
#   for j in range(0, 12):
#     query = """ SELECT body, author, subreddit, TIMESTAMP_SECONDS(created_utc) FROM fh-bigquery.reddit_comments.""" + year + """ WHERE author IN ("""+ f1 +""") AND subreddit NOT IN ("""+ f4 +""")"""
#     print(query)
#     x = client.query(query).to_dataframe()
#     print("The query for the month", year)
#     if len(x) > 0: 
#       print(x)
#       text = pd.concat([text,x])
#     else:
#       print("Empty Dataframe")
  
#     print("\n\n")
    
#     month = str(int(year[-2:]) + 1)
#     if int(month) < 10:
#       month = '0'+month
#     year = year[0:-2]+month
#     if year == "2018_11":
#       break


# text = text.drop(['timestamp'], axis = 1)
# text = text.rename(columns={'f0_': 'timestamp'})
# text.to_csv("/content/gdrive/MyDrive/CommentsOfTopHaters.csv", index = False)

In [43]:
haters_comments = pd.read_csv("/content/gdrive/MyDrive/CommentsOfTopHaters.csv")
haters_comments

,body,author,subreddit,timestamp
0,Op if you're still here I can help you,lets_get_hyyerr,gokarts,2015-01-27 17:48:33+00:00
1,I am happy to hear you are on a smooth part of...,Captian_Cocksmith,stopdrinking,2015-01-09 12:19:11+00:00
2,I really wish there was somewhere closer than ...,erokk88,lexington,2015-01-31 01:34:07+00:00
3,Sounds like a stupid plan. But raining trash ...,theresheblowshard,explainlikeimfive,2015-01-19 03:49:35+00:00
4,"You can easily find them online, and if your n...",lets_get_hyyerr,gokarts,2015-01-27 17:50:46+00:00
...,...,...,...,...
190475,Now you’re just making up strawmen. I never sa...,Greatmambojambo,WhitePeopleTwitter,2018-10-27 05:01:15+00:00
190476,"Yes, exactly that. By being a member of a grou...",Greatmambojambo,WhitePeopleTwitter,2018-10-27 03:39:06+00:00
190477,What a fucking idiot.,Greatmambojambo,WhitePeopleTwitter,2018-10-28 06:58:06+00:00
190478,None of that makes any fucking sense,HighOnGoofballs,WhitePeopleTwitter,2018-10-29 18:52:15+00:00


In [44]:
# Classifier has been run and results have been saved
# !python2 /content/gdrive/MyDrive/Classifier/classifier/classifier.py

In [45]:
classified_haters_comments = pd.read_csv("/content/gdrive/MyDrive/ClassifiedHatersComments.csv")
classified_haters_comments = classified_haters_comments.sort_values('timestamp')
classified_haters_comments

,body,author,subreddit,timestamp,predictions
2114,"Eh, she's ok I guess. I just find it hard to b...",bottomlines,unitedkingdom,2015-01-01 01:35:04+00:00,Neither
731,Actually no. I don't want them to do things di...,bottomlines,apple,2015-01-01 01:57:56+00:00,Neither
2418,Has 4chan even done anything besides a few peo...,AdmiralAkbar1,TumblrInAction,2015-01-01 02:17:08+00:00,Neither
431,"Well, he took you this far as freshman, I expe...",smackythefrog,CFB,2015-01-01 03:12:50+00:00,Neither
396,"I don't know if you truly believe this, but yo...",smackythefrog,CFB,2015-01-01 03:22:31+00:00,Offensive language
...,...,...,...,...,...
189725,"lol damn, that's exactly who I was thinking it...",smackythefrog,TumblrInAction,2018-10-31 23:48:34+00:00,Neither
189133,Theres a pretty big difference between closin...,RamblingMutt,PoliticalHumor,2018-10-31 23:50:09+00:00,Neither
189175,So are you going to die with a high point 9mm ...,RamblingMutt,PoliticalHumor,2018-10-31 23:52:32+00:00,Neither
190246,This post is with the lord now for the followi...,Idontreadrepliesnoob,oldpeoplefacebook,2018-10-31 23:54:37+00:00,Neither


In [46]:
ol = classified_haters_comments[classified_haters_comments['predictions'] == "Offensive language"]
ol

,body,author,subreddit,timestamp,predictions
396,"I don't know if you truly believe this, but yo...",smackythefrog,CFB,2015-01-01 03:22:31+00:00,Offensive language
424,"Eight hours later, I am a goddamn retard.... I...",smackythefrog,CFB,2015-01-01 06:12:19+00:00,Offensive language
801,Like am I seeing shit or visuals of what is sm...,lets_get_hyyerr,trees,2015-01-01 07:41:51+00:00,Offensive language
1194,WHO THE FUCK SAID THAT?,AdmiralAkbar1,AskReddit,2015-01-01 19:34:42+00:00,Offensive language
416,Golden Globe nominations doesn't mean shit.,homefree122,CFB,2015-01-02 02:57:12+00:00,Offensive language
...,...,...,...,...,...
185802,Gotta respect its purr-vacy.,smackythefrog,quityourbullshit,2018-10-30 06:25:46+00:00,Offensive language
186482,I have some in my fridge,ghostmetalblack,fakehistoryporn,2018-10-30 11:16:18+00:00,Offensive language
189272,"Uh huh, and I'm sure it was totally that simpl...",RamblingMutt,PoliticalHumor,2018-10-31 00:03:30+00:00,Offensive language
186650,Das ist eine ziemlich extreme Ansicht,Greatmambojambo,de,2018-10-31 11:48:31+00:00,Offensive language


In [47]:
ol['author'].unique()

array(['smackythefrog', 'lets_get_hyyerr', 'AdmiralAkbar1', 'homefree122',
       'erokk88', 'bottomlines', 'Captian_Cocksmith', 'KBDs',
       'Idontreadrepliesnoob', 'PedroIsWatching', 'HoundDogs',
       'Auphor_Phaksache', 'theresheblowshard', 'RamblingMutt',
       'Suzookus', 'wmegenney', 'HighOnGoofballs', 'Greatmambojambo',
       'RonDeGrasseDawtchins', 'Mechanical-one', 'wizardofthefuture',
       'G_Petronius', 'ArseKraken', 'FeralHushpuppy', 'CantGrammarGood',
       'ghostmetalblack'], dtype=object)

In [115]:
def find_timeline(name, df):
  df = df[df['author']==name]
  
  date = df['timestamp'].str[:7]
  df['date'] = date

  print("\nThese are comments made on subreddits that were not in either DB or DH.\n")
  print("Community participation of redditor:", name)
  subs = df['subreddit'].unique()
  print("Total number of subreddits commented on (other than DB and DH):", len(subs))
  print('\n')

  for i in df['date'].unique():
    month = df[df['date'] == i]
    r1 = random.randint(0, len(month)-1)
    r2 = random.randint(0, len(month)-1)
    print("Comment made in the month", i)
    print("Comment made on the subreddit:", month['subreddit'].iloc[r1])
    print("Comment classified as:", month['predictions'].iloc[r1])
    print(month['body'].iloc[r1])
    print('\n')
    print("Comment made on the subreddit:", month['subreddit'].iloc[r2])
    print("Comment classified as:", month['predictions'].iloc[r2])
    print(month['body'].iloc[r2])
    print("\n\n")

In [117]:
find_timeline('Mechanical-one', ol, )


These are comments made on subreddits that were not in either DB or DH.

Community participation of redditor: Mechanical-one
Total number of subreddits commented on (other than DB and DH): 43


Comment made in the month 2016-01
Comment made on the subreddit: AskReddit
Comment classified as: Offensive language
In high school first year I tool all applied courses. Second year I jumped up to academic because why not. 

One teacher. A math teacher. This fucking bitch I tell you. Said someone who took applied the year before would not do well in her class. Said it while looking at me and other students who had jumped up. I was the only one still there next class the rest had switched out. 

This fucking bitch had it out for.me I swear. She b
Never explained anything properly and if you went up to ask for help she would say you should have listened and tell you to go do the work. 

Ended up failing and taking summer school. Same material  and did an 80. She fucked me over and failed me beca

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
